In [ ]:
# Check GPU Specs
!nvidia-smi

In [ ]:
# Get hashes to crack
# Make a directory called 'dothashcat' in your Google Drive before this
# with the 'hashes' subdirectory for storing hashes
from google.colab import drive

# This will ask you to go to a link and get an authorization code
# to let the .hashcat folder get synced to your Google Drive
drive.mount('/content/drive')

# Make a symbolic link between Google Drive
# and the local .hashcat directory
!ls /content/drive/My\ Drive/dothashcat
!ln -s /content/drive/My\ Drive/dothashcat /root/.hashcat

In [ ]:
# Clone, build and install Hashcat
!apt install cmake build-essential -y
!apt install checkinstall git -y
!git clone https://github.com/hashcat/hashcat.git
!cd hashcat && git submodule update --init && make && make install

In [ ]:
# Check the Hashcat Version to make sure it's installed correctly
!hashcat --version

In [ ]:
# WPA2 EAPOL Benchmark (change this to check the speed of whichever hash you're cracking)
!hashcat -w 4 -m 2500 --benchmark

In [ ]:
# Get wordlists
!mkdir /root/wordlists && cd /root/wordlists

# Clone the SecLists repository
!git clone https://github.com/danielmiessler/SecLists /root/wordlists/SecLists

# Append 'hashcat' for demonstration purposes
!echo "hashcat!" >> /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou-75.txt

!ls /root/wordlists/SecLists

In [ ]:
# Check that your hashes are accessible to Colab
!ls /root/.hashcat/hashes

In [ ]:
# Start pure brute force cracking with a session name
# !hashcat -m 2500 -a3 --session $SESSION /root/.hashcat/hashes/test.hccapx
# !hashcat -m 2500 /root/.hashcat/hashes/test.hccapx -a 3 -w 4 --session test-session

# Restore the session
# !hashcat --session $SESSION --restore

# Crack with a wordlist
!hashcat -m 2500 /root/.hashcat/hashes/test.hccapx /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou-75.txt

In [ ]:
# creating new directory to store cap files
!mkdir cap_files
# uploading cap
from google.colab import files

uploaded = files.upload()

for f_name in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=f_name, length=len(uploaded[f_name])))

!mv {f_name} "/content/cap_files/{f_name}"

In [ ]:
import glob
import subprocess
import re
from tqdm import tqdm
is_cracked = False
wordlists = list()
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Common-Credentials/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Cracked-Hashes/milw0rm-dictionary.txt'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Default-Credentials/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Honeypot-Captures/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Leaked-Databases/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Malware/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/WiFi-WPA/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Software/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/*.txt'):
  wordlists.append(wordlist)

In [ ]:
def check_if_cracked(command):
  output = subprocess.check_output([command + " --show"], shell=True).decode()
  if len(output) == 0:
    return False
  else:
    return True



In [ ]:
cap_file = f"/content/cap_files/{f_name} "
for wordlist in tqdm(wordlists):
  command = 'hashcat -m 2500 ' + cap_file + wordlist
  subprocess.call([command],shell=True)
  if check_if_cracked(command):
    is_cracked = True
  if is_cracked:
    print("Password is cracked!!")
    cracked_password = subprocess.check_output([command + " --show"], shell=True).decode()
    cracked_password = cracked_password.split(':')[-1]
    print(f"Password : {cracked_password}")
    print(f"wordlist used : {wordlist}")
    break
  else:
    continue

if is_cracked == False:
  print("Password was not found in SecLists")
else:
  pass